In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
print(os.listdir("../input/imet-2019-fgvc6"))

# Any results you write to the current directory are saved as output.

import cv2
from multiprocessing import Pool
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T
import math
import torch.nn.functional as F
import random
from PIL import Image
from collections import OrderedDict

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
#     tf.set_random_seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
seed_everything(42)
DATASET_PATH = '../input/imet-2019-fgvc6'

In [ ]:
train_data = pd.read_csv(os.path.join(DATASET_PATH, 'train.csv'))
train_data['attribute_ids'] = train_data['attribute_ids'].apply(lambda s: [int(x) for x in s.split()])

classes = pd.read_csv(os.path.join(DATASET_PATH, 'labels.csv'))
id_to_class = {row['attribute_id']: row['attribute_name'] for _, row in classes.iterrows()}
class_to_id = {id_to_class[k]: k for k in id_to_class}

In [ ]:
# def load_shape(id):
#     return cv2.imread(os.path.join('../input/train/{}.png'.format(id))).shape

# with Pool(os.cpu_count()) as pool:
#     sizes = pool.map(load_shape, tqdm(train_data['id']))

# del x
# id = train_data.id.values[0]
# im = 
# print(im.shape)

# _, x = cv2.imencode('.jpg', im)
# print(im.size)
# print(x.shape)

# def tmp(id):
#     image = cv2.imread(os.path.join(DATASET_PATH, 'train/{}.png'.format(id)))
#     image = cv2.resize(image, (128, 128))
#     image = cv2.imencode('.jpg', image)

# x = [tmp(id) for id in tqdm(train_data.id)]
# x = [Image.open(os.path.join(DATASET_PATH, 'train/{}.png'.format(id))) for id in tqdm(train_data.id)]
# print(len(x), x[0])

In [ ]:
# train_data['height'] = [size[0] for size in sizes]
# train_data['width'] = [size[1] for size in sizes]
# train_data['ratio'] = train_data['width'] / train_data['height']

# # plt.scatter(train_data['width'], train_data['height'])
# print(train_data[['width', 'height', 'ratio']].describe())

# print('min', train_data['ratio'].idxmin())
# print('max', train_data['ratio'].idxmax())
# print(train_data

In [ ]:
# plt.figure(figsize=(200,200))
# plt.imshow(cv2.imread('../input/train/{}.png'.format(train_data.loc[47121]['id'])))

In [ ]:
# ls = train_data['attribute_ids'].apply(lambda xs: [id_to_class[x] for x in xs])


# print(labels.head())
# # ls = labels.apply(lambda xs: [x for x in xs if x.startswith('culture')])

# print(ls.head())

# ls = ls.apply(len)
# print(len(ls))
# print(ls.hist(bins=100))

# print(ls.value_counts())

# # print(labels[(ls == 4).values])

In [ ]:
class TrainEvalDataset(torch.utils.data.Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        row = self.data.iloc[i]
        
        image = load_image(os.path.join(DATASET_PATH, 'train/{}.png'.format(row['id'])))
        if self.transform is not None:
            image = self.transform(image)

        label = np.zeros(NUM_CLASSES, dtype=np.float32)
        for l in row['attribute_ids']: # TODO:
            label[l] = 1.
            
        return image, label, row['id']   
    
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, transform=None):
        self.data = os.listdir(os.path.join(DATASET_PATH, 'test'))
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        path = self.data[i]
        id = os.path.splitext(path)[0]
        
        image = load_image(os.path.join(DATASET_PATH, 'test', path))
        if self.transform is not None:
            image = self.transform(image)
        
        return image, id
    
# def load_image(path, size):
#     image = cv2.imread(path)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(image, size)
    
#     return image

# def load_image(path, size):
#     image = Image.open(path)
#     image = image.resize(size)
#     image = np.array(image)
    
#     return image

def load_image(path):
    image = Image.open(path)
    
    return image

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        if ARCH == 'resnet34':
            block = torchvision.models.resnet.BasicBlock
            self.model = ResNet(block, [3, 4, 6, 3])
            self.model.load_state_dict(torch.load('../input/resnet34/resnet34.pth'))
            self.model.fc = nn.Linear(512 * block.expansion, NUM_CLASSES)
        elif ARCH == 'resnet50':
            block = torchvision.models.resnet.Bottleneck
            self.model = ResNet(block, [3, 4, 6, 3])
            self.model.load_state_dict(torch.load('../input/resnet50/resnet50.pth'))
            self.model.fc = nn.Linear(512 * block.expansion, NUM_CLASSES)
        elif ARCH == 'seresnext50':
            block = SEResNeXtBottleneck
            self.model = SENet(
                block, [3, 4, 6, 3], groups=32, reduction=16, dropout_p=None, 
                inplanes=64, input_3x3=False, downsample_kernel_size=1, downsample_padding=0,
                num_classes=1000)
            self.model.load_state_dict(torch.load('../input/torch-models/se_resnext50_32x4d.pth'))
            self.model.last_linear = nn.Linear(512 * block.expansion, NUM_CLASSES)
        else:
            raise AssertionError('invalid ARCH {}'.format(ARCH))

    def forward(self, input):
        return self.model(input)
    
class Attention(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        
        self.weight = nn.Conv2d(in_features, 1, kernel_size=1)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
    
    def forward(self, input):
        b, c, h, w = input.size()
        
        weight = self.weight(input)
        weight = weight.view(b, 1, h * w)
        weight = weight.softmax(-1)

        input = input.view(b, c, h * w)
        input = input * weight
        input = input.sum(-1)
        input = input.view(b, c, 1, 1)
        
        return input
    
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion))

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x
    
class SENet(nn.Module):
    def __init__(self, block, layers, groups, reduction, dropout_p=0.2, inplanes=128, input_3x3=True, 
                 downsample_kernel_size=3, downsample_padding=1, num_classes=1000):
        super(SENet, self).__init__()
        
        self.inplanes = inplanes
        if input_3x3:
            layer0_modules = [
                ('conv1', nn.Conv2d(3, 64, 3, stride=2, padding=1, bias=False)),
                ('bn1', nn.BatchNorm2d(64)),
                ('relu1', nn.ReLU(inplace=True)),
                ('conv2', nn.Conv2d(64, 64, 3, stride=1, padding=1, bias=False)),
                ('bn2', nn.BatchNorm2d(64)),
                ('relu2', nn.ReLU(inplace=True)),
                ('conv3', nn.Conv2d(64, inplanes, 3, stride=1, padding=1, bias=False)),
                ('bn3', nn.BatchNorm2d(inplanes)),
                ('relu3', nn.ReLU(inplace=True)),
            ]
        else:
            layer0_modules = [
                ('conv1', nn.Conv2d(3, inplanes, kernel_size=7, stride=2, padding=3, bias=False)),
                ('bn1', nn.BatchNorm2d(inplanes)),
                ('relu1', nn.ReLU(inplace=True)),
            ]
        # To preserve compatibility with Caffe weights `ceil_mode=True`
        # is used instead of `padding=1`.
        layer0_modules.append(('pool', nn.MaxPool2d(3, stride=2, ceil_mode=True)))
        self.layer0 = nn.Sequential(OrderedDict(layer0_modules))
        self.layer1 = self._make_layer(
            block,
            planes=64,
            blocks=layers[0],
            groups=groups,
            reduction=reduction,
            downsample_kernel_size=1,
            downsample_padding=0)
        self.layer2 = self._make_layer(
            block,
            planes=128,
            blocks=layers[1],
            stride=2,
            groups=groups,
            reduction=reduction,
            downsample_kernel_size=downsample_kernel_size,
            downsample_padding=downsample_padding)
        self.layer3 = self._make_layer(
            block,
            planes=256,
            blocks=layers[2],
            stride=2,
            groups=groups,
            reduction=reduction,
            downsample_kernel_size=downsample_kernel_size,
            downsample_padding=downsample_padding)
        self.layer4 = self._make_layer(
            block,
            planes=512,
            blocks=layers[3],
            stride=2,
            groups=groups,
            reduction=reduction,
            downsample_kernel_size=downsample_kernel_size,
            downsample_padding=downsample_padding)
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(dropout_p) if dropout_p is not None else None
        self.last_linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, blocks, groups, reduction, stride=1,
                    downsample_kernel_size=1, downsample_padding=0):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(
                    self.inplanes, planes * block.expansion, kernel_size=downsample_kernel_size, 
                    stride=stride, padding=downsample_padding, bias=False),
                nn.BatchNorm2d(planes * block.expansion))

        layers = []
        layers.append(block(self.inplanes, planes, groups, reduction, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups, reduction))

        return nn.Sequential(*layers)

    def features(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        return x

    def logits(self, x):
        x = self.avg_pool(x)
        if self.dropout is not None:
            x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.last_linear(x)
        
        return x

    def forward(self, x):
        x = self.features(x)
        x = self.logits(x)
        
        return x
    
class Bottleneck(nn.Module):
    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out = self.se_module(out) + residual
        out = self.relu(out)

        return out
    
class SEResNeXtBottleneck(Bottleneck):
    expansion = 4

    def __init__(self, inplanes, planes, groups, reduction, stride=1,
                 downsample=None, base_width=4):
        super(SEResNeXtBottleneck, self).__init__()
        
        width = math.floor(planes * (base_width / 64)) * groups
        self.conv1 = nn.Conv2d(inplanes, width, kernel_size=1, bias=False, stride=1)
        self.bn1 = nn.BatchNorm2d(width)
        self.conv2 = nn.Conv2d(width, width, kernel_size=3, stride=stride, padding=1, groups=groups, bias=False)
        self.bn2 = nn.BatchNorm2d(width)
        self.conv3 = nn.Conv2d(width, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.se_module = SEModule(planes * 4, reduction=reduction)
        self.downsample = downsample
        self.stride = stride
        
class SEModule(nn.Module):
    def __init__(self, channels, reduction):
        super(SEModule, self).__init__()
        
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Conv2d(channels, channels // reduction, kernel_size=1, padding=0)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(channels // reduction, channels, kernel_size=1, padding=0)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        module_input = x
        
        x = self.avg_pool(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        
        return module_input * x

In [ ]:
class OneCycleScheduler(object):
    def __init__(self, optimizer, lr, beta, max_steps, annealing):
        self.optimizer = optimizer
        self.lr = lr
        self.beta = beta
        self.max_steps = max_steps
        self.annealing = annealing
        self.epoch = -1
    
    def step(self):
        self.epoch += 1
        
        mid = round(self.max_steps * 0.3)
        if self.epoch < mid:
            r = self.epoch / mid
            lr = self.annealing(self.lr[0], self.lr[1], r)
            beta = self.annealing(self.beta[0], self.beta[1], r)
        else:
            r = (self.epoch - mid) / (self.max_steps - mid)
            lr = self.annealing(self.lr[1], self.lr[0] / 1e4, r)
            beta = self.annealing(self.beta[1], self.beta[0], r)

        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr
            
            if 'betas' in param_group:
                param_group['betas'] = (beta, *param_group['betas'][1:])
            elif 'momentum' in param_group:
                param_group['momentum'] = beta
            else:
                raise AssertionError('no beta parameter')

def annealing_linear(start, end, r):
    return start + r * (end - start)            
            
def annealing_cos(start, end, r):
    cos_out = np.cos(np.pi * r) + 1
    return end + (start - end) / 2 * cos_out   

In [ ]:
class Mean(object):
    def __init__(self):
        self.values = []

    def compute(self):
        return sum(self.values) / len(self.values)

    def update(self, value):
        self.values.extend(np.reshape(value, [-1]))

    def reset(self):
        self.values = []

    def compute_and_reset(self):
        value = self.compute()
        self.reset()
        
        return value
    
class EWA(object):
    def __init__(self, beta=0.9):
        self.beta = beta
        self.step = 0
        self.average = 0
        
    def update(self, value):
        self.step += 1
        self.average = self.beta * self.average + (1 - self.beta) * value
        
    def compute(self):
        return self.average / (1 - self.beta ** self.step)

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2):
        super().__init__()
        self.gamma = gamma

    def forward(self, input, target):
        target = target.float()
        max_val = (-input).clamp(min=0)
        loss = input - input * target + max_val + \
               ((-max_val).exp() + (-input - max_val).exp()).log()

        invprobs = F.logsigmoid(-input * (target * 2.0 - 1.0))
        loss = (invprobs * self.gamma).exp() * loss
        if len(loss.size())==2:
            loss = loss.sum(dim=1)
        return loss.mean()

def f2_loss(input, target, eps=1e-7):
    input = input.sigmoid()

    tp = (target * input).sum(1)
    tn = ((1 - target) * (1 - input)).sum(1)
    fp = ((1 - target) * input).sum(1)
    fn = (target * (1 - input)).sum(1)

    p = tp / (tp + fp + eps)
    r = tp / (tp + fn + eps)
    
    beta_sq = 2**2
    f2 = (1 + beta_sq) * p * r / (beta_sq * p + r + eps)
    loss = -(f2 + eps).log()
    
    return loss

def hinge_loss(input, target, delta=1.):
    target = target * 1. + (1 - target) * -1.
    loss = torch.max(torch.zeros_like(input).to(input.device), delta - target * input)
    
    return loss

def compute_loss(input, target):
    loss = [l(input=input, target=target).mean() for l in LOSS]
    loss = sum(loss) / len(loss)
    
    return loss

def compute_score(input, target, threshold=0.5):
    input = (input.sigmoid() > threshold).float()

    tp = (target * input).sum(-1)
    tn = ((1 - target) * (1 - input)).sum(-1)
    fp = ((1 - target) * input).sum(-1)
    fn = (target * (1 - input)).sum(-1)

    p = tp / (tp + fp)
    r = tp / (tp + fn)
    
    beta_sq = 2**2
    f2 = (1 + beta_sq) * p * r / (beta_sq * p + r)
    f2[f2 != f2] = 0.
    
    return f2

def find_threshold_global(input, target):
    thresholds = np.arange(0.1, 0.9, 0.01)
    scores = [compute_score(input=input, target=target, threshold=t).mean() 
              for t in tqdm(thresholds, desc='threshold search')]
    threshold = thresholds[np.argmax(scores)]
    score = scores[np.argmax(scores)]
    
    plt.plot(thresholds, scores)
    plt.axvline(threshold)
    plt.title('score: {:.8f}, threshold: {:.8f}'.format(score.item(), threshold))
    plt.show()
    
    return threshold

# def find_threshold_class(input, target, initial):
#     threshold = torch.full((NUM_CLASSES, ), initial).to(input.device)
    
#     tmp = []
#     for _ in range(5):
#         for i in tqdm(range(NUM_CLASSES), desc='threshold search'):
#             space = np.arange(0.1, 0.9, 0.01)
#             scores = []

#             for t in space:
#                 threshold[i] = t
#                 score = compute_score(input=input, target=target, threshold=threshold).mean() 
#                 scores.append(score)

#             threshold[i] = space[np.argmax(scores)]
#             score = scores[np.argmax(scores)]
#         tmp.append(score)
#         print(score)
#     print(tmp)
    
#     plt.hist(threshold.cpu(), bins=50)
#     plt.title('score: {:.8f}, threshold mean: {:.8f}, threshold std: {:.8f}'.format(score.item(), threshold.mean(), threshold.std()))
#     plt.show()
    
#     return threshold

def find_threshold_class(input, target, initial):
    threshold = torch.full((NUM_CLASSES, ), initial).to(input.device)
    steps = []
    for _ in tqdm(range(50), desc='threshold search'):
        for i in np.random.permutation(NUM_CLASSES):
            r = torch.tensor([threshold[i] - 0.01, threshold[i], threshold[i] + 0.01])
            space = threshold.view(1, NUM_CLASSES).repeat(r.size(0), 1)
            space[:, i] = r
            scores = compute_score(input=input.unsqueeze(0), target=target.unsqueeze(0), threshold=space.unsqueeze(1))
            scores = scores.mean(-1)
            threshold[i] = r[scores.argmax()]
            
        score = compute_score(input=input, target=target, threshold=threshold).mean()
        steps.append(score)
        
    plt.plot(steps)
    plt.show()
    
    plt.hist(threshold.cpu(), bins=50)
    plt.title('score: {:.8f}, threshold mean: {:.8f}, threshold std: {:.8f}'.format(score.item(), threshold.mean(), threshold.std()))
    plt.show()
    
    return threshold

In [ ]:
EPOCHS = 5
NUM_CLASSES = len(classes)
BATCH_SIZE = 256
WORKERS = 0
ANNEAL = annealing_linear
ARCH = 'seresnext50'
OPT = 'adam'
AUG = 'med'
IMAGE_SIZE = 128
LOSS_SMOOTHING = 0.9
LOSS = [
#     f2_loss,
#     F.binary_cross_entropy_with_logits,
    FocalLoss(),
#     hinge_loss,
]

# TODO: pin memory
# TODO: stochastic weight averaging
# TODO: group images by buckets (size, ratio) and batch
# TODO: hinge loss clamp instead of minimum
# TODO: losses
# TODO: better one cycle
# TODO: cos vs lin
# TODO: load and restore state after lr finder
# TODO: better loss smoothing
# TODO: shuffle thresh search
# TODO: init thresh search from global best
# TODO: shuffle split
# TODO: tune on large size
# TODO: cross val
# TODO: smart sampling
# TODO: larger model
# TODO: imagenet papers
# TODO: load image as jpeg
# TODO: augmentations (flip, crops, color)
# TODO: min 1 tag?
# TODO: pick threshold to match ratio
# TODO: compute smoothing beta from batch size and num steps
# TODO: speedup image loading
# TODO: pin memory
# TODO: smart sampling
# TODO: better threshold search (step, epochs)
# TODO: weight standartization
# TODO: label smoothing
# TODO: search threshold for each class
# TODO: build sched for lr find

In [ ]:
to_tensor_and_norm = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

if AUG == 'low':
    train_transform = T.Compose([
        T.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        T.RandomHorizontalFlip(),
        to_tensor_and_norm,
    ])
    eval_transform = T.Compose([
        T.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        to_tensor_and_norm,
    ])
elif AUG == 'med':
    train_transform = T.Compose([
        T.RandomResizedCrop((IMAGE_SIZE, IMAGE_SIZE), scale=(1., 1.), ratio=(3. / 4., 4. / 3.)),
        T.RandomHorizontalFlip(),
        to_tensor_and_norm,
    ])
    eval_transform = T.Compose([
        T.Resize(IMAGE_SIZE),
        T.CenterCrop(IMAGE_SIZE),
        to_tensor_and_norm,
    ])
elif AUG == 'med+color':
    train_transform = T.Compose([
        T.RandomResizedCrop((IMAGE_SIZE, IMAGE_SIZE), scale=(1., 1.), ratio=(3. / 4., 4. / 3.)),
        T.RandomHorizontalFlip(),
        T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
        to_tensor_and_norm,
    ])
    eval_transform = T.Compose([
        T.Resize(IMAGE_SIZE),
        T.CenterCrop(IMAGE_SIZE),
        to_tensor_and_norm,
    ])
elif AUG == 'hard':
    scale = (0.6, 1.0)
    image_size_corrected = round(IMAGE_SIZE * (1 / np.mean(scale).item()))
        
    train_transform = T.Compose([
        T.RandomResizedCrop((IMAGE_SIZE, IMAGE_SIZE), scale=scale, ratio=(3. / 4., 4. / 3.)),
        T.RandomHorizontalFlip(),
        to_tensor_and_norm,
    ])
    eval_transform = T.Compose([
        T.Resize(image_size_corrected),
        T.CenterCrop(image_size_corrected),
        to_tensor_and_norm,
    ])
else:
    raise AssertionError('invalid AUG {}'.format(AUG))

s = len(train_data)

train_dataset = TrainEvalDataset(train_data.iloc[s//5:], transform=train_transform)
train_data_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=BATCH_SIZE, drop_last=True, shuffle=True, num_workers=WORKERS) # TODO: all args

eval_dataset = TrainEvalDataset(train_data.iloc[:s//5], transform=eval_transform)
eval_data_loader = torch.utils.data.DataLoader(
    eval_dataset, batch_size=BATCH_SIZE, num_workers=WORKERS) # TODO: all args

test_dataset = TestDataset(transform=eval_transform)
test_data_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=BATCH_SIZE, num_workers=WORKERS) # TODO: all args

### <<<
def build_optimizer(parameters, lr, weight_decay):
    if OPT == 'adam':
        return torch.optim.Adam(parameters, lr, weight_decay=weight_decay)
    elif OPT == 'sgd':
        return torch.optim.SGD(parameters, lr, momentum=0.9, weight_decay=weight_decay)
    else:
        raise AssertionError('invalid OPT {}'.format(OPT))

def set_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

min_lr = 1e-8
max_lr = 10.
p = (max_lr / min_lr)**(1 / len(train_data_loader))

stp = 0
cur_lr = min_lr
lrs = []
lss = []
sls = []
ewa = EWA(beta=LOSS_SMOOTHING)

minima = {
    'loss': np.inf,
    'lr': min_lr
}

device = torch.device('cuda:0')
model = Model()
model = model.to(device)
optimizer = build_optimizer(model.parameters(), min_lr, weight_decay=1e-4)

model.train()
for images, labels, ids in tqdm(train_data_loader, desc='lr search'):
    images, labels = images.to(device), labels.to(device)
    logits = model(images)

    loss = compute_loss(input=logits, target=labels)

    ### <<<
    ewa.update(loss.data.cpu().numpy().mean())
    lrs.append(cur_lr)
    lss.append(loss.data.cpu().numpy().mean())
    sls.append(ewa.compute())
    if ewa.compute() < minima['loss']:
        minima['loss'] = ewa.compute()
        minima['lr'] = cur_lr
    if minima['loss'] * 4 < ewa.compute():
        break
    stp += 1

    cur_lr = min_lr * p**stp
    set_lr(optimizer, cur_lr)
    ### <<<

    optimizer.zero_grad()
    loss.mean().backward()
    optimizer.step()
    
#     break # <<<<<<<<<<<<<<

plt.plot(lrs, lss)
plt.plot(lrs, sls)
plt.axvline(minima['lr'])
plt.xscale('log')
plt.title('loss: {:.8f}, lr: {:.8f}'.format(minima['loss'], minima['lr']))
plt.show()
### <<<

del model
del optimizer
del images, labels, logits, loss

device = torch.device('cuda:0')
model = Model()
model = model.to(device)
optimizer = build_optimizer(model.parameters(), minima['lr'] / 10, weight_decay=1e-4)
scheduler = OneCycleScheduler(
    optimizer, lr=(minima['lr'] / 10 / 25, minima['lr'] / 10), beta=(0.95, 0.85), max_steps=len(train_data_loader) * EPOCHS, annealing=ANNEAL)

metrics = {
    'loss': Mean(),
    'score': Mean()
}

stats = {
    'train_loss': [],
    'eval_loss': [],
    'eval_score': []
}

best_score = 0
for epoch in range(EPOCHS):
    model.train()
    for images, labels, ids in tqdm(train_data_loader, desc='epoch {} train'.format(epoch)):
        images, labels = images.to(device), labels.to(device)
        logits = model(images)

        loss = compute_loss(input=logits, target=labels)
        metrics['loss'].update(loss.data.cpu().numpy())
        
        scheduler.step()
        optimizer.zero_grad()
        loss.mean().backward()
        optimizer.step()
        
#         break # <<<<<<<<<
        
    loss = metrics['loss'].compute_and_reset()
    stats['train_loss'].append(loss)
    print('[TRAIN] loss: {:.8f}'.format(loss))    
        
    model.eval()
    with torch.no_grad():
        for images, labels, ids in tqdm(eval_data_loader, desc='epoch {} evaluation'.format(epoch)):
            images, labels = images.to(device), labels.to(device)
            logits = model(images)

            loss = compute_loss(input=logits, target=labels)
            metrics['loss'].update(loss.data.cpu().numpy())
            score = compute_score(input=logits, target=labels)
            metrics['score'].update(score.data.cpu().numpy())
            
#             break # <<<<<<<<<
            
    loss = metrics['loss'].compute_and_reset()
    stats['eval_loss'].append(loss)
    score = metrics['score'].compute_and_reset()
    stats['eval_score'].append(score)
    print('[EVAL] loss: {:.8f}, score: {:.8f}'.format(loss, score))
    
    if score > best_score:
        best_score = score
        torch.save(model.state_dict(), './model.pth')
        
#     break # <<<<<<<<<<

plt.plot(stats['train_loss'])
plt.plot(stats['eval_loss'])
plt.show()
plt.plot(stats['eval_score'])
plt.show()
        
model.load_state_dict(torch.load('./model.pth'))
predictions = []
targets = []
model.eval()
with torch.no_grad():
    for images, labels, ids in tqdm(eval_data_loader, desc='best model evaluation'.format(epoch)):
        images, labels = images.to(device), labels.to(device)
        logits = model(images)
        
        targets.append(labels)
        predictions.append(logits)
        
#         break # <<<<<<<<

    del images, labels, logits

    predictions = torch.cat(predictions, 0)
    targets = torch.cat(targets, 0)
    threshold = find_threshold_global(input=predictions, target=targets)
#     threshold = find_threshold_class(input=predictions, target=targets, initial=threshold)

submission = []
model.eval()
with torch.no_grad():
    for images, ids in tqdm(test_data_loader, desc='inference'):
        images = images.to(device)
        logits = model(images)

        for id, logits in zip(ids, logits):
            pred = (logits.sigmoid() > threshold).nonzero().reshape(-1)
            pred = pred.data.cpu().numpy()
            pred = map(str, pred)
            pred = ' '.join(pred)

            submission.append((id, pred))
            
submission = pd.DataFrame(submission, columns=['id', 'attribute_ids'])
submission.to_csv('./submission.csv', index=False)